# # Yellow Detection
# 
# This script sets up a camera feed and processes the frames to detect yellow lanes.
# It adjusts the car's steering based on the detected lane's position.


In [ ]:

import ipywidgets
import traitlets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg
import cv2
import numpy as np


In [ ]:

# Importing the CSICamera class from the jetcam library to initialize and control the camera.
from jetcam.csi_camera import CSICamera

# Initialize the camera with a resolution of 224x224.
camera = CSICamera(width=224, height=224)


In [ ]:

# Start the camera feed.
camera.running = True


In [ ]:

# Import the NvidiaRacecar class to control the car.
from jetracer.nvidia_racecar import NvidiaRacecar
car = NvidiaRacecar()


# Define the HSV range for the yellow color
# 
# Setting the lower and upper bounds for the yellow color in HSV format to detect yellow lanes.


In [ ]:

LOWER_YELLOW_HSV = np.array([11, 80, 87])
UPPER_YELLOW_HSV = np.array([22, 136, 182])


# Smoothing factor for center point calculation
# 
# ALPHA is a smoothing factor used to average the current center point with the previous center point for smoother steering.


In [ ]:

ALPHA = 0.2


# Initialize previous center point
# 
# `prev_center` keeps track of the previous center point of the detected lane for smoothing purposes.


In [ ]:

prev_center = None


In [ ]:

# Create an image widget to display the contours.
contour_widget = ipywidgets.Image(format='jpeg', width=camera.width, height=camera.height)


# Create image preview widget, Display the camera widget
# 
# Setting up the widget to display the camera feed in the Jupyter notebook.


In [ ]:

camera_widget = ipywidgets.Image(width=camera.width, height=camera.height)
traitlets.dlink((camera, 'value'), (camera_widget, 'value'), transform=bgr8_to_jpeg)
display(camera_widget)


# # Display sliders for steering gain, bias, and throttle
# 
# Creating sliders for adjusting the car's steering gain, bias, and throttle in real-time.


In [ ]:

steering_gain_slider = ipywidgets.FloatSlider(description='Steering Gain', min=-1.0, max=1.0, value=-0.7, step=0.01, orientation='horizontal', layout={'width': '300px'})
steering_bias_slider = ipywidgets.FloatSlider(description='Steering Bias', min=-0.5, max=0.5, value=0.0, step=0.01, orientation='horizontal', layout={'width': '300px'})
steering_value_slider = ipywidgets.FloatSlider(description='Steering', min=-1.0, max=1.0, value=0, step=0.01, orientation='horizontal', disabled=False, layout={'width': '400px'})
throttle_slider = ipywidgets.FloatSlider(description='Throttle', min=-1.0, max=1.0, value=0.15, step=0.01, orientation='vertical')

# Link sliders to car properties
steering_gain_link = traitlets.link((steering_gain_slider, 'value'), (car, 'steering_gain'))
steering_offset_link = traitlets.link((steering_bias_slider, 'value'), (car, 'steering_offset'))
throttle_slider_link = traitlets.link((throttle_slider, 'value'), (car, 'throttle'))

# Display the sliders and camera widgets in a horizontal box layout.
display(ipywidgets.HBox([
    ipywidgets.VBox([steering_gain_slider, steering_bias_slider, steering_value_slider]),
    throttle_slider, camera_widget, contour_widget
]))


# Function to map value ranges
# 
# Helper function to map a value from one range to another. This is used to map the detected center point to steering values.


In [ ]:

def map_value(value, from_low, from_high, to_low, to_high):
    if value is None:
        return None
    return (value - from_low) * (to_high - to_low) / (from_high - from_low) + to_low


# Function to process frame and detect lane
# 
# Main function to process each frame from the camera. It detects yellow lanes, calculates the center of the lanes, and draws the lane lines and center point on the frame.


In [ ]:

import cv2
import numpy as np

# Define the lower and upper bounds for yellow color in HSV
LOWER_YELLOW_HSV = np.array([20, 100, 100])
UPPER_YELLOW_HSV = np.array([30, 255, 255])

# Parameters for center smoothing
ALPHA = 0.5
prev_center = None

def process_frame(frame):
    global prev_center

    # Convert frame to HSV color space
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Threshold the HSV image to get only yellow colors
    yellow_mask = cv2.inRange(hsv, LOWER_YELLOW_HSV, UPPER_YELLOW_HSV)

    # Apply Gaussian blur to smooth the mask
    blurred = cv2.GaussianBlur(yellow_mask, (5, 5), 0)

    # Define the region of interest (ROI)
    height, width, _ = frame.shape
    roi_vertices = [
        (0, height),
        (0, height // 2),
        (width, height // 2),
        (width, height)
    ]
    mask = np.zeros_like(blurred)
    cv2.fillPoly(mask, np.array([roi_vertices], np.int32), 255)
    masked_blurred = cv2.bitwise_and(blurred, mask)

    # Edge detection using Canny
    edges = cv2.Canny(masked_blurred, 50, 150)

    # Hough Line Transform to detect lines
    lines = cv2.HoughLinesP(edges, 1, np.pi / 180, threshold=50, minLineLength=50, maxLineGap=20)

    # Create a blank image with the same dimensions as the original frame
    lane_image = np.copy(frame)

    # Initialize lists to store the coordinates of the leftmost and rightmost lines
    left_lines = []
    right_lines = []

    # Initialize center coordinates
    center_x = None
    center_y = None

    # Draw the lines on the blank image and store the leftmost and rightmost lines
    if lines is not None:
        for line in lines:
            for x1, y1, x2, y2 in line:
                if y1 > height // 2 and y2 > height // 2:  # Filter lines in the ROI
                    if x1 < width // 2 and x2 < width // 2:
                        left_lines.append((x1, y1, x2, y2))
                    elif x1 > width // 2 and x2 > width // 2:
                        right_lines.append((x1, y1, x2, y2))

    # Draw the leftmost line
    if left_lines:
        left_line = sorted(left_lines, key=lambda line: line[0])[0]
        cv2.line(lane_image, (left_line[0], left_line[1]), (left_line[2], left_line[3]), (0, 255, 0), 5)

    # Draw the rightmost line
    if right_lines:
        right_line = sorted(right_lines, key=lambda line: line[0])[-1]
        cv2.line(lane_image, (right_line[0], right_line[1]), (right_line[2], right_line[3]), (0, 255, 0), 5)

    # Fill the area between the leftmost and rightmost lines
    if left_lines and right_lines:
        pts = np.array([
            [left_line[0], left_line[1]],
            [left_line[2], left_line[3]],
            [right_line[2], right_line[3]],
            [right_line[0], right_line[1]]
        ])
        cv2.fillPoly(lane_image, [pts], (0, 255, 0))

        # Calculate the center point between the two lines
        center_x = (left_line[0] + left_line[2] + right_line[0] + right_line[2]) // 4
        center_y = (left_line[1] + left_line[3] + right_line[1] + right_line[3]) // 4

        # Smoothing the center point
        if prev_center is not None:
            center_x = int(ALPHA * center_x + (1 - ALPHA) * prev_center[0])
            center_y = int(ALPHA * center_y + (1 - ALPHA) * prev_center[1])

        prev_center = (center_x, center_y)

        # Ensure the center point is within the ROI
        if center_y > height // 2:
            # Draw a circle at the center point
            cv2.circle(lane_image, (center_x, center_y), 15, (0, 0, 255), -1)
    else:
        if prev_center is not None:
            center_x, center_y = prev_center

    # Extract and draw contours
    contours, _ = cv2.findContours(yellow_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contour_image = np.zeros_like(frame)
    cv2.drawContours(contour_image, contours, -1, (0, 255, 0), 2)

    # Calculate the center of the detected lanes
    if contours:
        bounding_boxes = [cv2.boundingRect(contour) for contour in contours]
        x_centers = [(x + w // 2) for x, y, w, h in bounding_boxes]
        y_centers = [(y + h // 2) for x, y, w, h in bounding_boxes]

        if x_centers and y_centers:
            avg_x = int(sum(x_centers) / len(x_centers))
            avg_y = int(sum(y_centers) / len(y_centers))
            cv2.circle(contour_image, (avg_x, avg_y), 10, (0, 0, 255), -1)
            center_coordinates = (avg_x, avg_y)
        else:
            center_coordinates = (0, 0)
    else:
        center_coordinates = (0, 0)

    return lane_image, contour_image, center_coordinates


# Function to update the car's steering based on the center point
# 
# This function updates the car's steering value based on the center point of the detected lane.
# It uses the `map_value` function to convert the center point to a steering value and updates the car's steering.


In [ ]:

def update_frame(change):
    frame = change['new']
    processed_frame, another, center = process_frame(frame)

    if center[0] is not None:
        x = map_value(center[0], 0, camera.width, -1.0, 1.0)
        steering = (x * steering_gain_slider.value + steering_bias_slider.value) * -1
        steering = max(min(steering, 1.0), -1.0)
        steering_value_slider.value = steering
        car.steering = steering
        contour_widget.value = bgr8_to_jpeg(another)

    camera_widget.value = bgr8_to_jpeg(processed_frame)


In [ ]:

# Observe changes in the camera feed and call `update_frame` function to process each frame.
camera.observe(update_frame, names='value')
